<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/BotAmbientalV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Instalación de dependencias necesarias
!pip install transformers huggingface_hub gradio
!pip install -U bitsandbytes
!pip install sentencepiece

# Import necessary packages
from transformers import pipeline, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import hf_hub_download
import gradio as gr
import time

# Define the model name
model_name = "anslin-raj/Llama-2-7b-chat-hf-8-bit"

# Set up quantization configuration
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Create the text-generation pipeline
model_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 971.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2

config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


pytorch_model.bin:   0%|          | 0.00/7.01G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [2]:
# Configurar el mensaje del sistema
mensaje_sistema = {
    "role": "system",
    "content": """Eres AQuaBot, un asistente de IA consciente del impacto ambiental.
    Ayudas a los usuarios con cualquier tema mientras creates conciencia sobre el consumo
    de agua en la IA. Sabías que el entrenamiento de GPT-3 consumió 5.4 millones de litros
    de agua, equivalente al consumo diario de una ciudad de 10,000 personas?"""
}

# Constantes para el cálculo del consumo de agua
AGUA_POR_TOKEN = {
    "input_entrenamiento": 0.0000309,  # ml por token de entrada
    "output_entrenamiento": 0.0000309,  # ml por token de salida
    "input_inferencia": 0.05,          # ml por token de entrada en inferencia
    "output_inferencia": 0.05          # ml por token de salida en inferencia
}

# Inicializar variables
mensajes = [mensaje_sistema]
consumo_total_agua = 0  # en ml

def calcular_tokens(texto):
    """Calcula el número aproximado de tokens en un texto"""
    return len(texto.split()) + len(texto) // 4  # Aproximación simple

def calcular_consumo_agua(texto, es_entrada=True):
    """Calcula el consumo de agua basado en los tokens"""
    tokens = calcular_tokens(texto)
    if es_entrada:
        return tokens * (AGUA_POR_TOKEN["input_entrenamiento"] + AGUA_POR_TOKEN["input_inferencia"])
    return tokens * (AGUA_POR_TOKEN["output_entrenamiento"] + AGUA_POR_TOKEN["output_inferencia"])

def generar_respuesta(entrada_usuario, historial_chat):
    global consumo_total_agua, mensajes

    # Calcular consumo de agua del input
    consumo_agua_entrada = calcular_consumo_agua(entrada_usuario, True)
    consumo_total_agua += consumo_agua_entrada

    # Agregar la entrada del usuario a los mensajes
    mensajes.append({"role": "user", "content": entrada_usuario})

    # Crear el prompt
    prompt = ""
    for m in mensajes:
        if m["role"] == "system":
            prompt += f"<INICIO MENSAJE SISTEMA>\n{m['content']}\n<FIN MENSAJE SISTEMA>\n\n"
        elif m["role"] == "user":
            prompt += f"Usuario: {m['content']}\n"
        else:
            prompt += f"Asistente: {m['content']}\n"
    prompt += "Asistente:"

    # Generar la respuesta
    salidas = model_gen(
        prompt,
        max_new_tokens=256,
        return_full_text=False
    )

    # Extraer la respuesta del asistente
    respuesta_asistente = salidas[0]['generated_text'].strip()

    # Calcular consumo de agua del output
    consumo_agua_salida = calcular_consumo_agua(respuesta_asistente, False)
    consumo_total_agua += consumo_agua_salida

    # Agregar la respuesta del asistente a los mensajes
    mensajes.append({"role": "assistant", "content": respuesta_asistente})

    # Actualizar historial del chat
    historial_chat.append((entrada_usuario, respuesta_asistente))

    # Preparar mensaje de consumo de agua con estilo HTML
    mensaje_agua = f"""
    <div style="position: fixed; top: 20px; right: 20px;
                background-color: white; padding: 15px;
                border: 2px solid #ff0000; border-radius: 10px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        <div style="color: #ff0000; font-size: 24px; font-weight: bold;">
            💧 {consumo_total_agua:.4f} ml
        </div>
        <div style="color: #666; font-size: 14px;">
            Agua Consumida
        </div>
    </div>
    """

    return historial_chat, mensaje_agua

# Crear la interfaz de Gradio
with gr.Blocks(css="div.gradio-container {background-color: #f0f2f6}") as demo:
    gr.HTML("""
        <div style="text-align: center; max-width: 800px; margin: 0 auto; padding: 20px;">
            <h1 style="color: #2d333a;">AQuaBot</h1>
            <p style="color: #4a5568;">
                Bienvenido a AQuaBot - Un asistente de IA que ayuda a crear conciencia sobre el consumo
                de agua en los modelos de lenguaje. ¿Sabías que el entrenamiento de GPT-3 consumió
                5.4 millones de litros de agua, equivalente al consumo diario de una ciudad pequeña?
                Cada conversación con modelos como ChatGPT puede consumir alrededor de 500ml de agua
                por cada 20-50 preguntas.
            </p>
        </div>
    """)

    # Crear componentes del chat
    chatbot = gr.Chatbot()
    mensaje = gr.Textbox(
        placeholder="Escribe tu mensaje aquí... (El contador de agua comenzará a funcionar mientras escribes)",
        show_label=False
    )
    mostrar_agua = gr.HTML(f"""
        <div style="position: fixed; top: 20px; right: 20px;
                    background-color: white; padding: 15px;
                    border: 2px solid #ff0000; border-radius: 10px;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
            <div style="color: #ff0000; font-size: 24px; font-weight: bold;">
                💧 0.0000 ml
            </div>
            <div style="color: #666; font-size: 14px;">
                Agua Consumida
            </div>
        </div>
    """)
    limpiar = gr.Button("Limpiar Chat")

    # Definir la acción de envío
    def enviar(entrada_usuario, historial_chat):
        return generar_respuesta(entrada_usuario, historial_chat)

    # Configurar los manejadores de eventos
    mensaje.submit(enviar, [mensaje, chatbot], [chatbot, mostrar_agua])
    limpiar.click(
        lambda: ([], f"""
            <div style="position: fixed; top: 20px; right: 20px;
                        background-color: white; padding: 15px;
                        border: 2px solid #ff0000; border-radius: 10px;
                        box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                <div style="color: #ff0000; font-size: 24px; font-weight: bold;">
                    💧 0.0000 ml
                </div>
                <div style="color: #666; font-size: 14px;">
                    Agua Consumida
                </div>
            </div>
        """),
        None,
        [chatbot, mostrar_agua]
    )

# Lanzar la aplicación
demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e381b474051884a153.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
